Data source: http://web.mta.info/developers/developer-data-terms.html#data

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
agency = pd.read_csv('gtfs/agency.txt')
calendar_dates = pd.read_csv('gtfs/calendar_dates.txt')
calendar = pd.read_csv('gtfs/calendar.txt')
routes = pd.read_csv('gtfs/routes.txt')
shapes = pd.read_csv('gtfs/shapes.txt')
stop_times = pd.read_csv('gtfs/stop_times.txt')
stops = pd.read_csv('gtfs/stops.txt')
trips = pd.read_csv('gtfs/trips.txt')

In [3]:
service_trip_id = pd.merge(calendar[['service_id']],trips[['service_id', 'trip_id', 'route_id']])
service_trip_id.head()

,service_id,trip_id,route_id
0,1,7659520,5
1,1,7659523,5
2,1,7659519,5
3,1,7659515,5
4,1,7659517,5


In [4]:
calendar = pd.merge(calendar,service_trip_id)
calendar.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,trip_id,route_id
0,1,0,0,0,0,0,1,1,20170331,20170930,7659520,5
1,1,0,0,0,0,0,1,1,20170331,20170930,7659523,5
2,1,0,0,0,0,0,1,1,20170331,20170930,7659519,5
3,1,0,0,0,0,0,1,1,20170331,20170930,7659515,5
4,1,0,0,0,0,0,1,1,20170331,20170930,7659517,5


### Add abritrary arrival and departure dates to timetables

Define a function to add arrival dates to stop_times tables. We will use 1/5/15 as the start date because that is the date we are currently using to visualize taxi trips. This code deals with arrival times > 24 hours and converts them to 1/6/15.

In [23]:
arbitrary_dates = ['1/5/2015', '1/6/2015']

In [7]:
def add_arrival_date(df, dates = arbitrary_dates):
    df = df.copy()
    arrival_date = []
    arrival_time = []

    for i in df['arrival_time']:
        hour = i[:i.find(':')]
        minute = i[i.find(':')+1:i.find(':',4)]
        second = i[i.find(':',5)+1:]
        
        if int(hour) < 24:
            arrival_date.append(dates[0])
            arrival_time.append(i)
        elif 24 <= int(hour) < 48:
            arrival_date.append(dates[1])
            hour = int(hour) - 24
            arrival_time.append(str(hour)+":"+minute+":"+second)
        else:
            arrival_date.append('NA')
            
    df['arrival_date'] = arrival_date
    df['arrival_time'] = arrival_time
    return df

In [8]:
def add_departure_date(df, dates = arbitrary_dates):
    df = df.copy()
    departure_date = []
    departure_time = []

    for i in df['departure_time']:
        hour = i[:i.find(':')]
        minute = i[i.find(':')+1:i.find(':',4)]
        second = i[i.find(':',5)+1:]
        
        if int(hour) < 24:
            departure_date.append(dates[0])
            departure_time.append(i)
        elif 24 <= int(hour) < 48:
            departure_date.append(dates[1])
            hour = int(hour) - 24
            departure_time.append(str(hour)+":"+minute+":"+second)
        else:
            departure_date.append('NA')
            
    df['departure_date'] = departure_date
    df['departure_time'] = departure_time
    return df

In [9]:
monday = calendar[calendar['monday'] == 1]
tuesday = calendar[calendar['tuesday'] == 1]
wednesday = calendar[calendar['wednesday'] == 1]
thursday = calendar[calendar['thursday'] == 1]
friday = calendar[calendar['friday'] == 1]
saturday = calendar[calendar['saturday'] == 1]
sunday = calendar[calendar['sunday'] == 1]

## Monday

In [11]:
monday_trips = monday['trip_id']
print "There are ", len(monday_trips), "trips on Monday"
monday_trips.head()

There are  718 trips on Monday


472    7694524
473    7694525
474    7694519
475    7694517
476    7694514
Name: trip_id, dtype: int64

In [12]:
triplist = []
count = 0

for i in monday_trips:
    count += 1
    df = stop_times[stop_times['trip_id'] == i]
    
    # add arrival and departure dates
    df = add_arrival_date(df, dates = arbitrary_dates)
    df = add_departure_date(df, dates = arbitrary_dates)
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])

    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction and route id
    try:
        df = pd.merge(df, trips[['trip_id', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')
    except:
        df = pd.merge(df, trips[['trip_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')
    
    # join df with routes to get route id
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')
    
    # create new dataframe to store results
    legs = pd.DataFrame()
    legs['type_id'] = df['route_id']
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = ((legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')).astype(int)
    legs['start_lat'] = df['stop_lat']
    legs['start_lon'] = df['stop_lon']
    legs['end_lat'] = legs['start_lat'].shift(-1).fillna(method='ffill')
    legs['end_lon'] = legs['start_lon'].shift(-1).fillna(method='ffill')
    
    # append results to triplist
    triplist.append(legs)
    
    if count % 100 == 0:
        print str(dt.datetime.now()), "finished trip number", count, "/", len(monday_trips)

2017-04-08 12:11:15.932016 finished trip number 100 / 718
2017-04-08 12:11:20.327738 finished trip number 200 / 718
2017-04-08 12:11:24.442798 finished trip number 300 / 718
2017-04-08 12:11:30.160014 finished trip number 400 / 718
2017-04-08 12:11:35.139980 finished trip number 500 / 718
2017-04-08 12:11:39.500210 finished trip number 600 / 718
2017-04-08 12:11:43.614887 finished trip number 700 / 718


In [13]:
data = pd.concat(triplist)

In [14]:
data = data[data.tripduration != 0]

In [15]:
data = data[data.start_lon != 0]

In [16]:
data = data.sort_values(by='starttime')
data = data.reset_index(drop=True)

In [17]:
data.shape

(8240, 8)

In [18]:
data['type_id'] = 15 # 15 = MetroNorth

In [19]:
data.to_csv("metronorth_monday_raw.csv")

In [20]:
%%bash 

head metronorth_monday_raw.csv

,type_id,starttime,stoptime,tripduration,start_lat,start_lon,end_lat,end_lon
0,15,2015-01-05 00:07:00,2015-01-05 00:17:00,600,40.752998,-73.977056,40.805157,-73.939149
1,15,2015-01-05 00:10:00,2015-01-05 00:20:00,600,40.752998,-73.977056,40.805157,-73.939149
2,15,2015-01-05 00:17:00,2015-01-05 00:54:00,2220,40.805157,-73.939149,41.046611,-73.542846
3,15,2015-01-05 00:18:00,2015-01-05 00:22:00,240,41.046611,-73.542846,41.070547,-73.520021
4,15,2015-01-05 00:18:00,2015-01-05 00:21:00,180,41.413203,-73.623787,41.39447,-73.619802
5,15,2015-01-05 00:20:00,2015-01-05 00:44:00,1440,40.805157,-73.939149,41.032589,-73.775208
6,15,2015-01-05 00:20:00,2015-01-05 00:30:00,600,40.752998,-73.977056,40.805157,-73.939149
7,15,2015-01-05 00:21:00,2015-01-05 00:26:00,300,41.39447,-73.619802,41.347722,-73.662269
8,15,2015-01-05 00:22:00,2015-01-05 00:25:00,180,41.070547,-73.520021,41.08876,-73.517828


In [21]:
%%bash

awk -F"," -v OFS="," '{ $1="" ; $0=substr($0,2) } 1' metronorth_monday_raw.csv > metronorth_monday.csv

In [22]:
%%bash 

head metronorth_monday.csv

type_id,starttime,stoptime,tripduration,start_lat,start_lon,end_lat,end_lon
15,2015-01-05 00:07:00,2015-01-05 00:17:00,600,40.752998,-73.977056,40.805157,-73.939149
15,2015-01-05 00:10:00,2015-01-05 00:20:00,600,40.752998,-73.977056,40.805157,-73.939149
15,2015-01-05 00:17:00,2015-01-05 00:54:00,2220,40.805157,-73.939149,41.046611,-73.542846
15,2015-01-05 00:18:00,2015-01-05 00:22:00,240,41.046611,-73.542846,41.070547,-73.520021
15,2015-01-05 00:18:00,2015-01-05 00:21:00,180,41.413203,-73.623787,41.39447,-73.619802
15,2015-01-05 00:20:00,2015-01-05 00:44:00,1440,40.805157,-73.939149,41.032589,-73.775208
15,2015-01-05 00:20:00,2015-01-05 00:30:00,600,40.752998,-73.977056,40.805157,-73.939149
15,2015-01-05 00:21:00,2015-01-05 00:26:00,300,41.39447,-73.619802,41.347722,-73.662269
15,2015-01-05 00:22:00,2015-01-05 00:25:00,180,41.070547,-73.520021,41.08876,-73.517828
